In [1]:
import pandas as pd 
data =  pd.read_csv('SMSSpamCollection', sep = '\t', names = ['label', 'message'])

In [2]:
text = data['message']
label = data['label']

In [3]:
data['word_count'] = data['message'].apply(lambda x: len(str(x).split(" ")))

In [4]:
data['char_count'] = data['message'].str.len() ## this also includes spaces
data[['message','char_count']].head()

,message,char_count
0,"Go until jurong point, crazy.. Available only ...",111
1,Ok lar... Joking wif u oni...,29
2,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,U dun say so early hor... U c already then say...,49
4,"Nah I don't think he goes to usf, he lives aro...",61


In [5]:
def avg_word(sentence):
  words = sentence.split()
  #print(words)
  return (sum(len(word) for word in words)/len(words))

data['avg_word'] = data['message'].apply(lambda x: avg_word(x))

In [6]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

data['stopwords'] = data['message'].apply(lambda x: len([x for x in x.split() if x in stop]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
data['hastags'] = data['message'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))

In [8]:
data['numerics'] = data['message'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))

In [9]:
data['upper'] = data['message'].apply(lambda x: len([x for x in x.split() if x.isupper()]))

In [10]:
pos_family = {
    'noun' : ['NN','NNS','NNP','NNPS'],
    'pron' : ['PRP','PRP$','WP','WP$'],
    'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
    'adj' :  ['JJ','JJR','JJS'],
    'adv' : ['RB','RBR','RBS','WRB']
}

# function to check and get the part of speech tag count of a words in a given sentence
from textblob import TextBlob, Word, Blobber
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
def check_pos_tag(x, flag):
    cnt = 0
    try:
        wiki = TextBlob(x)
        for tup in wiki.tags:
            ppo = list(tup)[1]
            if ppo in pos_family[flag]:
                cnt += 1
    except:
        pass
    return cnt

data['noun_count'] = data['message'].apply(lambda x: check_pos_tag(x, 'noun'))
data['verb_count'] = data['message'].apply(lambda x: check_pos_tag(x, 'verb'))
data['adj_count'] = data['message'].apply(lambda x: check_pos_tag(x, 'adj'))
data['adv_count'] = data['message'].apply(lambda x: check_pos_tag(x, 'adv'))
data['pron_count'] = data['message'].apply(lambda x: check_pos_tag(x, 'pron'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [11]:
features = data[['word_count','char_count','avg_word','stopwords','hastags','numerics','upper','noun_count','verb_count','adj_count', 'adv_count', 'pron_count']]

In [13]:
import numpy as np
classes_list = ["ham","spam"]
label_index = data['label'].apply(classes_list.index)
label = np.asarray(label_index)
label

array([0, 0, 1, ..., 0, 0, 0])

In [14]:
import numpy as np
features_array = np.asarray(features)

In [15]:
import numpy as np
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features_array, label, test_size=0.33, random_state=90)

# svm

In [16]:
from sklearn.metrics import accuracy_score
from sklearn import metrics

from sklearn.svm import SVC
model_SVM = SVC()
model_SVM.fit(x_train, y_train)
y_pred_SVM = model_SVM.predict(x_test)
print("SVM")
print("Accuracy score =", accuracy_score(y_test, y_pred_SVM))
print(metrics.classification_report(y_test, y_pred_SVM))

SVM
Accuracy score = 0.9336595976073954
              precision    recall  f1-score   support

           0       0.95      0.97      0.96      1600
           1       0.78      0.68      0.73       239

    accuracy                           0.93      1839
   macro avg       0.87      0.83      0.84      1839
weighted avg       0.93      0.93      0.93      1839



#Naive Bayes

In [17]:
from sklearn.naive_bayes import GaussianNB
naive = GaussianNB()
naive.fit(x_train,y_train)
y_pred_naive = naive.predict(x_test)
print("Naive Bayes")
print("Accuracy score =", accuracy_score(y_test, y_pred_naive))
print(metrics.classification_report(y_test, y_pred_naive ))

Naive Bayes
Accuracy score = 0.9265905383360522
              precision    recall  f1-score   support

           0       0.97      0.95      0.96      1600
           1       0.69      0.79      0.74       239

    accuracy                           0.93      1839
   macro avg       0.83      0.87      0.85      1839
weighted avg       0.93      0.93      0.93      1839

